In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
import codecs
from IPython.display import HTML

credits = pd.read_csv('tmdb_5000_credits.csv')
movies= pd.read_csv('tmdb_5000_movies.csv')

movies.head()


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [11]:
# covert json to string in the dataset base on the column name
def convert(name: str):
    movies[name]=movies[name].apply(json.loads)
    for index,i in zip(movies.index,movies[name]):
        list1=[]
        for j in range(len(i)):
            list1.append((i[j]['name']))
        movies.loc[index,name]=str(list1)



# convert json to string for genres, keywords, production_companies, production_countries, spoken_languages, cast
convert('genres')
convert('keywords')
convert('production_companies')
convert('production_countries')
convert('spoken_languages')

# cover json to string for cast
credits['cast']=credits['cast'].apply(json.loads)
for index,i in zip(credits.index,credits['cast']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    credits.loc[index,'cast']=str(list1)
    
# covert json to string for crew and replace the column name to director
credits['crew']=credits['crew'].apply(json.loads)
def finc_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
credits['crew']=credits['crew'].apply(finc_director)
credits.rename(columns={'crew':'director'},inplace=True)

In [12]:
# natural join the two dataset
movies = movies.merge(credits,left_on='id',right_on='movie_id',how='left')
movies = movies[['id','original_title','genres','cast','vote_average','director','keywords']]

In [13]:
movies.iloc[0]

id                                                            19995
original_title                                               Avatar
genres            ['Action', 'Adventure', 'Fantasy', 'Science Fi...
cast              ['Sam Worthington', 'Zoe Saldana', 'Sigourney ...
vote_average                                                    7.2
director                                              James Cameron
keywords          ['culture clash', 'future', 'space war', 'spac...
Name: 0, dtype: object

In [14]:
movies.shape

(4803, 7)

In [15]:
movies.size

33621

In [16]:
movies.columns

Index(['id', 'original_title', 'genres', 'cast', 'vote_average', 'director',
       'keywords'],
      dtype='object')

In [17]:
movies.dtypes

id                  int64
original_title     object
genres             object
cast               object
vote_average      float64
director           object
keywords           object
dtype: object

## Handle the string data
reference: https://www.kaggle.com/code/heeraldedhia/movie-ratings-and-recommendation-using-knn#Similarity-between-movies

In [ ]:
#TODO

## KNN from scratch
reference: https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761

In [18]:
import math
from collections import Counter

def knn_scratch(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    
    for index, example in enumerate(data):
        distance = distance_fn(example[:-1], query)
        neighbor_distances_and_indices.append((distance, index))
    
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    k_nearest_labels = [data[i][1] for distance, i in k_nearest_distances_and_indices]
    
    return k_nearest_distances_and_indices, choice_fn(k_nearest_labels)


def mean(labels):
    return sum(labels) / len(labels)

def mode(labels):
    return Counter(labels).most_common(1)[0][0]

def euclidean_distance(point1, point2):
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

In [19]:
'''
    # Regression Data
    # 
    # Column 0: height (inches)
    # Column 1: weight (pounds)
    '''
reg_data = [
    [65.75, 112.99],
    [71.52, 136.49],
    [69.40, 153.03],
    [68.22, 142.34],
    [67.79, 144.30],
    [68.70, 123.30],
    [69.80, 141.49],
    [70.01, 136.46],
    [67.90, 112.37],
    [66.49, 127.45],
]

# Question:
# Given the data we have, what's the best-guess at someone's weight if they are 60 inches tall?
reg_query = [60]
reg_k_nearest_neighbors, reg_prediction = knn_scratch(
    reg_data, reg_query, k=3, distance_fn=euclidean_distance, choice_fn=mean
)


reg_k_nearest_neighbors, reg_prediction

([(5.75, 0), (6.489999999999995, 9), (7.790000000000006, 4)],
 128.24666666666667)

In [20]:
'''
# Classification Data
# 
# Column 0: age
# Column 1: likes pineapple
'''
clf_data = [
    [22, 1],
    [23, 1],
    [21, 1],
    [18, 1],
    [19, 1],
    [25, 0],
    [27, 0],
    [29, 0],
    [31, 0],
    [45, 0],
]
# Question:
# Given the data we have, does a 33 year old like pineapples on their pizza?
clf_query = [33]
clf_k_nearest_neighbors, clf_prediction = knn_scratch(
    clf_data, clf_query, k=3, distance_fn=euclidean_distance, choice_fn=mode
)

clf_k_nearest_neighbors, clf_prediction

([(2.0, 8), (4.0, 7), (6.0, 6)], 0)

In [32]:
from scipy import spatial

# knn for movies
def knn_movies(movie_query, k, column_names, similarity_fn, prediect_label, choice_fn):
    movie_similarity = []
    
    for index, movie in movies.iterrows():
        similarity = similarity_fn(movie, movie_query, column_names)
        movie_similarity.append((similarity, index))
    
    sorted_movie_similarity = sorted(movie_similarity)
    
    k_nearest_similarities = sorted_movie_similarity[:k]
    
    k_nearest_labels = [movies.iloc[i]['original_title'] for distance, i in k_nearest_similarities]
    
    return k_nearest_similarities, choice_fn(k_nearest_labels)


# euclidean distance function that handles movie's features list in column_names
def euclidean_distance_movie(movie1, movie2, column_names):
    distance = 0
    for name in column_names:
        distance += math.pow(int(movie1[name]) - (movie2[name]), 2)
    return math.sqrt(distance)

def choose_with_highest_vote_average(labels):
    return max(labels)

In [33]:
# test euclidean distance function with two movies with all features(genres, keywords, production_companies, production_countries, spoken_languages, cast, vote_average, director)
features = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages', 'cast', 'vote_average', 'director']
euclidean_distance_movie(movies.iloc[0], movies.iloc[1],features)

ValueError: invalid literal for int() with base 10: "['Action', 'Adventure', 'Fantasy', 'Science Fiction']"

In [23]:
movies.iloc[0], movies.iloc[1]

(id                                                            19995
 original_title                                               Avatar
 genres            ['Action', 'Adventure', 'Fantasy', 'Science Fi...
 cast              ['Sam Worthington', 'Zoe Saldana', 'Sigourney ...
 vote_average                                                    7.2
 director                                              James Cameron
 keywords          ['culture clash', 'future', 'space war', 'spac...
 Name: 0, dtype: object,
 id                                                              285
 original_title             Pirates of the Caribbean: At World's End
 genres                           ['Adventure', 'Fantasy', 'Action']
 cast              ['Johnny Depp', 'Orlando Bloom', 'Keira Knight...
 vote_average                                                    6.9
 director                                             Gore Verbinski
 keywords          ['ocean', 'drug abuse', 'exotic island', 'east...
 Name: 1,

0.9000000000000004

In [26]:
movies.iloc[0], movies.iloc[2]

(id                                                            19995
 original_title                                               Avatar
 genres            ['Action', 'Adventure', 'Fantasy', 'Science Fi...
 cast              ['Sam Worthington', 'Zoe Saldana', 'Sigourney ...
 vote_average                                                    7.2
 director                                              James Cameron
 keywords          ['culture clash', 'future', 'space war', 'spac...
 Name: 0, dtype: object,
 id                                                           206647
 original_title                                              Spectre
 genres                             ['Action', 'Adventure', 'Crime']
 cast              ['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...
 vote_average                                                    6.3
 director                                                 Sam Mendes
 keywords          ['spy', 'based on novel', 'secret agent', 'seq...
 Name: 2,